In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time, datetime

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)
%matplotlib inline

sp500 = pd.read_csv('Notebooks/Data/sp500.csv', index_col='Symbol', usecols=[0, 2, 3, 7])

In [2]:
sp500

                        Sector   Price  Book Value
Symbol                                            
MMM                Industrials  141.14      26.668
ABT                Health Care   39.60      15.573
ABBV               Health Care   53.95       2.954
ACN     Information Technology   79.79       8.326
ACE                 Financials  102.91      86.897
...                        ...     ...         ...
YHOO    Information Technology   35.02      12.768
YUM     Consumer Discretionary   74.77       5.147
ZMH                Health Care  101.84      37.181
ZION                Financials   28.43      30.191
ZTS                Health Care   30.53       2.150

[500 rows x 3 columns]

In [3]:
# Создадим датафрейм 10000 случайными числами
df = pd.DataFrame({'foo': np.random.random(10000), 'key': range(100, 10100)})
df[:5], df.index

(        foo  key
 0  0.023892  100
 1  0.279250  101
 2  0.947302  102
 3  0.071890  103
 4  0.655950  104,
 RangeIndex(start=0, stop=10000, step=1))

In [4]:
# Отбираем конкретную строку
df[df.key == 10099]

           foo    key
9999  0.812614  10099

In [5]:
# %timeit позволяет выполнить операцию 7 цекла по 1000 итераций
%timeit df[df.key==10099]


220 µs ± 8.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
# %timeit позволяет выполнить операцию 3 цекла по 1000 итераций
%timeit df.loc[9999] 

72.5 µs ± 795 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Превращаем столбец key в индекс - df.set_index(['key'])

In [7]:
df_with_index = df.set_index(['key'])
df_with_index[:5], df_with_index.index

(          foo
 key          
 100  0.023892
 101  0.279250
 102  0.947302
 103  0.071890
 104  0.655950,
 Int64Index([  100,   101,   102,   103,   104,   105,   106,   107,   108,
               109,
             ...
             10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098,
             10099],
            dtype='int64', name='key', length=10000))

In [8]:
# Теперь можно найти это значение при помощи индекса.
df_with_index.loc[10099]

foo    0.812614
Name: 10099, dtype: float64

## Типы индексов в пандас

In [9]:
# Покажем что столбцы фактически явл. индексами
temps = pd.DataFrame({
    "City": ["Missoula", "Philadelphia"],
    "Temperature": [70, 80]
})
temps

           City  Temperature
0      Missoula           70
1  Philadelphia           80

In [10]:
# Мы видим, что столбцы это индекс
temps.columns, '---', temps.index

(Index(['City', 'Temperature'], dtype='object'),
 '---',
 RangeIndex(start=0, stop=2, step=1))

### Индексы int64index и Rangeindex, в качестве меток использ. целые числа.

In [11]:
df_i64 = pd.DataFrame(np.arange(10, 20), index=np.arange(0, 10))
df_i64[:5], df_i64.index

(    0
 0  10
 1  11
 2  12
 3  13
 4  14,
 Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64'))

### Индекс Float64index, для числовых меток с плавающей точкой

In [12]:
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), np.arange(0.0, 100.0, 0.5))
df_f64, df_f64.index

(        0
 0.0     0
 0.5     5
 1.0    10
 1.5    15
 2.0    20
 ...   ...
 97.5  975
 98.0  980
 98.5  985
 99.0  990
 99.5  995
 
 [200 rows x 1 columns],
 Float64Index([ 0.0,  0.5,  1.0,  1.5,  2.0,  2.5,  3.0,  3.5,  4.0,  4.5,
               ...
               95.0, 95.5, 96.0, 96.5, 97.0, 97.5, 98.0, 98.5, 99.0, 99.5],
              dtype='float64', length=200))

### Использование дискретных интервалов с использованием - intervalIndex

In [13]:
df_interval = pd.DataFrame({"A": [1, 2, 3, 4]},
                          index=pd.IntervalIndex.from_breaks([0, 0.5, 1.0, 1.5, 2.0]))
df_interval, df_interval.index

(            A
 (0.0, 0.5]  1
 (0.5, 1.0]  2
 (1.0, 1.5]  3
 (1.5, 2.0]  4,
 IntervalIndex([(0.0, 0.5], (0.5, 1.0], (1.0, 1.5], (1.5, 2.0]], dtype='interval[float64, right]'))

### Категории в качестве индекса - CategoricalIndex

In [14]:
df_categorical = pd.DataFrame({'A': np.arange(6),
                              'B': list('aabbca')})

# Импортируем класс CategoricalDtype для работы с категорильными данными
from pandas.api.types import CategoricalDtype

df_categorical['B'] = df_categorical['B'].astype(CategoricalDtype(categories=list('cabd')))

# Превращаем категориальный столбец в индекс
df_categorical = df_categorical.set_index('B')

df_categorical.index

CategoricalIndex(['a', 'a', 'b', 'b', 'c', 'a'], categories=['c', 'a', 'b', 'd'], ordered=False, dtype='category', name='B')

In [15]:
# Ищем значения в категории 'a'
df_categorical.loc['a']

   A
B   
a  0
a  1
a  5

### Индексация по датам и времени с помощью - DatetimeIndex

DatetimeIndex используется для представления набора дат и времени. Он широко
применяется в данных, представленных в виде временных рядов, когда измерения для наблюдений берутся через определенные промежутки времени.

In [16]:
# Создаем Datetimeindex на основе диапазона дат
rmg = pd.date_range('5/1/2017', periods=5, freq='H')
ts = pd.Series(np.random.randn(len(rmg)), index=rmg)
ts

2017-05-01 00:00:00    1.191550
2017-05-01 01:00:00    0.544118
2017-05-01 02:00:00   -0.214543
2017-05-01 03:00:00   -1.250935
2017-05-01 04:00:00    0.766211
Freq: H, dtype: float64

In [17]:
ts.index

DatetimeIndex(['2017-05-01 00:00:00', '2017-05-01 01:00:00',
               '2017-05-01 02:00:00', '2017-05-01 03:00:00',
               '2017-05-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

### Индексация периодов времени с помощью - PeriodIndex

Кроме того, иногда нужно представить в виде меток индекса периоды времени
типа дня, месяца или года. Это очень похоже на работу с интервалами, но теперь
речь идет о периоде времени. 

In [18]:
periods = pd.PeriodIndex(['2017-1', '2017-2', '2017-3'], freq='M')
period_series = pd.Series(np.random.randn(len(periods)), index=periods)
period_series, period_series.index

(2017-01    1.288845
 2017-02   -2.476043
 2017-03    0.401959
 Freq: M, dtype: float64,
 PeriodIndex(['2017-01', '2017-02', '2017-03'], dtype='period[M]'))

### Создание и использование индекса в объекте Series или объекте DataFrame

Индексы можно создать либо явно, либо позволить библиотеке pandas создать их
неявно.

Явное создание индекса происходит, когда вы задаете индекс, используя
параметр index конструктора.


In [19]:
# Самостоятельно создадим индекс DatetimeIndex
date_times = pd.DatetimeIndex(pd.date_range('5/1/2017', periods=5, freq='H'))
date_times_2 = pd.date_range('05/08/22', periods=10, freq='D')

date_times, date_times_2

(DatetimeIndex(['2017-05-01 00:00:00', '2017-05-01 01:00:00',
                '2017-05-01 02:00:00', '2017-05-01 03:00:00',
                '2017-05-01 04:00:00'],
               dtype='datetime64[ns]', freq='H'),
 DatetimeIndex(['2022-05-08', '2022-05-09', '2022-05-10', '2022-05-11',
                '2022-05-12', '2022-05-13', '2022-05-14', '2022-05-15',
                '2022-05-16', '2022-05-17'],
               dtype='datetime64[ns]', freq='D'))

In [20]:
# Создаем объект датафрейм, использгя индекс
df_date_times = pd.DataFrame(np.arange(0, len(date_times)), index=date_times)

df_date_times

                     0
2017-05-01 00:00:00  0
2017-05-01 01:00:00  1
2017-05-01 02:00:00  2
2017-05-01 03:00:00  3
2017-05-01 04:00:00  4

In [21]:
# Создаем датафрейм с 2 столбцами
df = pd.DataFrame(
                [np.arange(10, 12), np.arange(12, 14)],
                 columns=list('ab'),
                 index=list('vw')
             )
df

    a   b
v  10  11
w  12  13

In [22]:
# У датафрейма обращение через [] возвращает столбец а не строку как у объекта серии.
df['a']

v    10
w    12
Name: a, dtype: int64

In [23]:
# Извлекаем СТРОКУ 'w' по метке
df.loc['w']

a    12
b    13
Name: w, dtype: int64

#### Поиск значений в серии с помощью оператора [] эквивалентен использованию свойства .loc[].


In [24]:
# Создаем серию
s = pd.Series(np.arange(0, 5), index=list('abcde'))

# Создаем срез по серии
s['b':'d'], s.loc['b':'d']

(b    1
 c    2
 d    3
 dtype: int64,
 b    1
 c    2
 d    3
 dtype: int64)

In [25]:
# Можно передать список меток
s[['a', 'c', 'e']], s.loc[['a', 'c', 'e']]

(a    0
 c    2
 e    4
 dtype: int64,
 a    0
 c    2
 e    4
 dtype: int64)

### Преобразование данных в индекс и получение данных из индекса

C помощью метода **.reset_index()** можно сбросить индекс датафрейма. Чаще всего
этот метод используют, когда нужно поместить содержимое индекса в один или
несколько столбцов.


In [26]:
# Исследуем несколько строк датафрейма sp500
sp500[:5]

                        Sector   Price  Book Value
Symbol                                            
MMM                Industrials  141.14      26.668
ABT                Health Care   39.60      15.573
ABBV               Health Care   53.95       2.954
ACN     Information Technology   79.79       8.326
ACE                 Financials  102.91      86.897

In [27]:
# Сбрасываем индекс и помещаем его в нужный нам столбец
index_moved_to_col = sp500.reset_index()

# Установим новый индекс - Sector
n = index_moved_to_col.set_index('Sector')
n[:5]

                       Symbol   Price  Book Value
Sector                                           
Industrials               MMM  141.14      26.668
Health Care               ABT   39.60      15.573
Health Care              ABBV   53.95       2.954
Information Technology    ACN   79.79       8.326
Financials                ACE  102.91      86.897

In [28]:
# Установим столбец в качестве индекса датафрейма (set_index)
index_moved_to_col.set_index('Sector')

# index_moved_to_col.index, index_moved_to_col[:5].index

                       Symbol   Price  Book Value
Sector                                           
Industrials               MMM  141.14      26.668
Health Care               ABT   39.60      15.573
Health Care              ABBV   53.95       2.954
Information Technology    ACN   79.79       8.326
Financials                ACE  102.91      86.897
...                       ...     ...         ...
Information Technology   YHOO   35.02      12.768
Consumer Discretionary    YUM   74.77       5.147
Health Care               ZMH  101.84      37.181
Financials               ZION   28.43      30.191
Health Care               ZTS   30.53       2.150

[500 rows x 3 columns]

### Переиндексация объекта 

#### Переиндексация по строкам

In [29]:
reindex = sp500.reindex(index=['MMM', 'ABBV', 'FOO'])

# из sp500 удаляются все записи которые не соответствуют новому индексу и
# добавляются те, которые отсутствовали в первоначальном.со значениями NaN

# Это хороший способ фильтрации на основе меток индексов. Создается НОВЫЙ датафрейм.
reindex

             Sector   Price  Book Value
Symbol                                 
MMM     Industrials  141.14      26.668
ABBV    Health Care   53.95       2.954
FOO             NaN     NaN         NaN

#### Переиндексация столбцов

In [30]:
# Меняем порядок столбцов
sp500.reindex(columns=['Price',
                        'Book Value',
                        'NewCol'])[:5]

         Price  Book Value  NewCol
Symbol                            
MMM     141.14      26.668     NaN
ABT      39.60      15.573     NaN
ABBV     53.95       2.954     NaN
ACN      79.79       8.326     NaN
ACE     102.91      86.897     NaN

### Иерархическая индексация - MultiIndex.

Иерархическая индексация – это инструмент библиотеки pandas, который позво-
ляет комбинировать использование двух или более индексов для каждой строки.

**Каждый из индексов в иерархическом индексе называется уровнем.**

In [31]:
# Сначала помещаем символы в столбец
reindexed = sp500.reset_index()

# Теперь индексируем датафрейм по столбцам  Sector и Symbol
multi_fi = reindexed.set_index(['Sector', 'Symbol'])
multi_fi[:5]


                                Price  Book Value
Sector                 Symbol                    
Industrials            MMM     141.14      26.668
Health Care            ABT      39.60      15.573
                       ABBV     53.95       2.954
Information Technology ACN      79.79       8.326
Financials             ACE     102.91      86.897

In [32]:
# Наш индекс - это multiIndex
type(multi_fi.index), len(multi_fi.index.levels)

(pandas.core.indexes.multi.MultiIndex, 2)

In [33]:
# Каждый уровень представляем отдельный объект index
multi_fi.index.levels[0], multi_fi.index.levels[1]

(Index(['Consumer Discretionary', 'Consumer Discretionary ', 'Consumer Staples',
        'Consumer Staples ', 'Energy', 'Financials', 'Health Care',
        'Industrials', 'Industries', 'Information Technology', 'Materials',
        'Telecommunications Services', 'Utilities'],
       dtype='object', name='Sector'),
 Index(['A', 'AA', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACE', 'ACN', 'ACT', 'ADBE',
        ...
        'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YHOO', 'YUM', 'ZION', 'ZMH',
        'ZTS'],
       dtype='object', name='Symbol', length=500))

In [34]:
# Количество различных секторов (уникальных)
len(multi_fi.index.levels[0])

13

### Значение самого индекса в определенном уровне для каждой строки можно извлечь при помощи - get_level_value

In [35]:
# Значение в уровне 0
multi_fi.index.get_level_values(0)

Index(['Industrials', 'Health Care', 'Health Care', 'Information Technology',
       'Financials', 'Health Care', 'Information Technology', 'Utilities',
       'Health Care', 'Financials',
       ...
       'Utilities', 'Information Technology', 'Information Technology',
       'Financials', 'Industrials', 'Information Technology',
       'Consumer Discretionary', 'Health Care', 'Financials', 'Health Care'],
      dtype='object', name='Sector', length=500)

### Чтобы посмотреть значения датафрейма с помощью иерархического индекса необходимо использовать МЕТОД  - xs

Индекс полученного датафрейма состоит из уровней, которые НЕ БЫЛИ ЗАДАНЫ,
в данном случае это Symbol. Уровни, для которых были указаны значения, удаляются из полученного индекса.


In [51]:
# Получаем все акции, которые имеют значение Industrials
# Обратить внимание, что индекс уровня 0 не выводится
multi_fi.xs('Industrials')[:5]


         Price  Book Value
Symbol                    
MMM     141.14      26.668
ALLE     52.46       0.000
APH      95.71      18.315
AVY      48.20      15.616
BA      132.41      19.870

#### Можно отбирать КОНКРЕТНУЮ ЗАПИСЬ, указывая метку и уровень индекса

In [37]:
multi_fi.xs('ALLE', level=1)

             Price  Book Value
Sector                        
Industrials  52.46         0.0

#### Чтобы предотвратить удаление уровней из результатов, можно воспольз. drop_levels=False

Отбираем строки, в которых индекс уровня 0 (Sector) имеет зн. Industrials, БЕЗ УДАЛЕНИЯ УРОВНЕЙ

In [38]:
multi_fi.xs('Industrials', drop_level=False)[:5]

                     Price  Book Value
Sector      Symbol                    
Industrials MMM     141.14      26.668
            ALLE     52.46       0.000
            APH      95.71      18.315
            AVY      48.20      15.616
            BA      132.41      19.870

Чтобы отобрать строку, используя иерархию индексов, можно связать вместе выховы xs для разных уровне.

xs - это метод ТОЛЬКО для чтения значений.

In [53]:
# скомбинируем уровни индексов
multi_fi.xs('Industrials').xs('UPS'), '-----', multi_fi.xs(('Industrials', 'UPS'))


(Price         102.73
 Book Value      6.79
 Name: UPS, dtype: float64,
 '-----',
 Price         102.73
 Book Value      6.79
 Name: (Industrials, UPS), dtype: float64)